In [ ]:
from src.rag.components.shared.io import IOManager

In [ ]:
from pathlib import Path

In [ ]:
output_path = Path.cwd().joinpath("datasets", "parsed_documents_with_embeddings")
io_manager = IOManager(output_path)

In [ ]:
io_manager.document_path.exists()

In [ ]:
chunk_size = 5
all_documents = []
for i in range(0, 20, chunk_size):
	documents = io_manager.load_documents(i, i + chunk_size)
	all_documents.extend(documents)

In [ ]:
# generate a random number between 0 and 20
import random
random_number = random.randint(0, 20)

In [ ]:
sample_doc = all_documents[random_number]

In [ ]:
nodes = sample_doc.convert_to_milvus()

In [ ]:
nodes[0].get("text")

In [ ]:
from src.rag.components.shared.databases.milvus import MilvusDatabase

In [ ]:
COLLECTION_NAME = "my_collection"

In [ ]:
milvus_client = MilvusDatabase(
    host="http://localhost:19530",
    vector_dimension=1024,
    collection_name=COLLECTION_NAME,
    token=""
)

In [ ]:
milvus_client.connect()

In [ ]:
milvus_client.create_index_if_not_exists()

In [ ]:
milvus_client.client.query(collection_name=COLLECTION_NAME, output_fields=["count(*)"])

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:

embedding_model_name = "intfloat/multilingual-e5-large"

In [ ]:
embedding_model = SentenceTransformer(embedding_model_name)

In [ ]:
test_questions = ["what is dependency ingestion?"]

In [ ]:
question_embeddings  = embedding_model.encode(test_questions, convert_to_tensor=True)

In [ ]:
question_embeddings = question_embeddings.cpu().numpy()



In [ ]:
milvus_client.connect()

In [ ]:
responses = milvus_client.client.search(
    collection_name=COLLECTION_NAME,
    data=question_embeddings,
    limit=5,
    output_fields=["text", "metadata"],
    params={"metric_type": "L2"}
)

In [ ]:
len(responses)

In [ ]:
responses

In [ ]:
for question, response in zip(test_questions, responses):
    print(f"Question: {question}")
    print("===" * 20)
    for result in response:
        print(result.id, result.distance, result.entity.get("text"))
        print(result.entity.get("metadata"))
        print("\n")
    print("===" * 20)

In [ ]:
### Need to come and chech the results and implement vector search.